In [ ]:
!nvidia-smi


Mon Dec  9 14:05:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   30C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install ipython numpy tqdm
# !pip install transformers[torch]
!pip install transformers==4.30.2

Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.30.2 whi

# Demo
This demo shows how to use DISC model with your own sentence input.

The script will first load the model and data processing module.
Then, it will run inference on the input sentence and output the detected idiom from the sentence.


In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.chdir('/content/drive/MyDrive/CCL/DISC')
print(os.getcwd())

/content/drive/MyDrive/CCL/DISC


In [4]:
from IPython.display import display, HTML
import torch
import numpy as np
from tqdm import tqdm
from src.utils.model_util import load_model_from_checkpoint
from src.model.read_comp_triflow import ReadingComprehensionDetector as DetectorMdl
from config import Config as config
from demo_helper.data_processor import DataHandler
from demo_helper.visualize_helper import simple_scoring_viz

## 1. Load model

In [5]:
data_handler = DataHandler(config)
detector_model= load_model_from_checkpoint(DetectorMdl, data_handler.config)


Loading Pre-trained Glove Embeddings...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers grea

Loading model from /content/drive/MyDrive/CCL/DISC/checkpoints/ReadComp_magpie_random_cross_attn-glove-char-pos-tri_latest.mdl
=> loading checkpoint '/content/drive/MyDrive/CCL/DISC/checkpoints/ReadComp_magpie_random_cross_attn-glove-char-pos-tri_latest.mdl'


/content/drive/MyDrive/CCL/DISC/src/utils/model_util.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename)


=> loaded checkpoint '/content/drive/MyDrive/CCL/DISC/checkpoints/ReadComp_magpie_random_cross_attn-glove-char-pos-tri_latest.mdl'


## 2. Set and prepare input sentences

In [6]:
sentences = [
    # The following examples are idioms
    'If you’re head over heels, you’re completely in love.',
    'If you keep someone at arm’s length, you avoid becoming friendly with them.',
    'If you’re a chip off the old block, you’re similar in some distinct way to your father or mother.',
    'He must face the music for his transgression.',
    'Getting fired turned out to be a blessing in disguise.',
    'I’m sorry but I just can’t seem to wrap my head around it.',
    'At the end of the day, it is you who will take the heat.',
    'At the end of the day, it is you who will take the responsibility.',
    'I don’t want to be Hayley’s friend anymore, she stabbed me in the back!',
    'Why not go to the post office on your way to the mall and kill two birds with one stone?',
    'Hey, I’m feeling pretty angry right now. I’m going to go blow off some steam.',
    'As a rule of thumb, you should usually pay for your date’s dinner, too.',
    'If you burn the candle at both ends, you work excessively hard, say, by keeping two jobs or by leading a busy social life in the evening.',
    # The following examples are similes
    'You were as brave as a lion.',
    'This house is as clean as a whistle.',
    "Sometimes you feel like a nut, sometimes you don't.",
    # Negative examples (no idioms)
    "We will also see which library is recommended to use on each occasion and the unique capabilities of each library."
]
pie = ["head over heels", "arm’s length", "chip off the old block", "face the music", "blessing in disguise",
       "wrap my head", "take the heat", "at the end of the day", "Hayley’s friend","kill two birds with one stone",
       "blow off some steam", "rule of thumb", "burn the candle at both ends", "brave as a lion","clean as a whistle",
       "like a nut", "on each occasion"]

In [13]:
sentences = ["We only use white hat, ethical search engine optimization techniques that will lead to long term success and unlike many SEO firms we will never use any tactics that have a risk of harming your site."]
pie = ["white hat"]

In [7]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [14]:
data = data_handler.prepare_input(sentences, pie)

## 3. Model inference

In [15]:
with torch.no_grad():
  pie_embedding = detector_model(data)
#     ys_ = detector_model(data)
#     probs = torch.nn.functional.softmax(ys_, dim=-1)
# ys_ = ys_.cpu().detach().numpy()
# probs = probs.cpu().detach().numpy()
# idiom_class_probs = probs[:, :, -1].tolist()
# predicts = np.argmax(ys_, axis=2)
pie_embedding = pie_embedding.cpu().detach().numpy()

In [16]:
pie_embedding.shape

(1, 512)

In [17]:
save_path = "/content/drive/MyDrive/CCL/DISC/embeddings/white_hat.pt"  # 定义保存路径
torch.save(pie_embedding, save_path)  # 保存为 .pt 文件
print(f"Phrase embeddings saved to {save_path}")

Phrase embeddings saved to /content/drive/MyDrive/CCL/DISC/embeddings/white_hat.pt


## 4. Extract output

In [15]:
ys_.shape

(17, 34, 5)

In [16]:
sentences_tkns = data['xs_bert'].cpu().detach().numpy().tolist()
sentences_tkns = [data_handler.tokenizer.convert_ids_to_tokens(s) for s in sentences_tkns]

In [17]:
print('Visualize Results by Scoring: ')
for i in range(len(sentences_tkns)):
    s = simple_scoring_viz(sentences_tkns[i], idiom_class_probs[i], 'YlGn')
    display(HTML(s))

Visualize Results by Scoring: 


In [18]:
predicts = predicts == 4
predicts = predicts.astype(float)
print('Visualize Results by Classification: ')
for i in range(len(sentences_tkns)):
    s = simple_scoring_viz(sentences_tkns[i], predicts[i], 'YlGn')
    display(HTML(s))

Visualize Results by Classification: 
